OGOP's Time-Distance Calculator

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink
import numpy as np
import os
import pandas as pd
import math
from itertools import product
import matplotlib.pyplot as plt
import csv
import scipy.stats as stats


# For ranged lists --- arr_1d = np.arange(0, 10, 2).tolist() --- start, end, iteration

def all_eom(v=None, u=None, a=None, s=None, t=None):
    knowns = [v, u, a, s, t]
    if knowns.count(None) != 2:
        raise ValueError("Provide exactly three known variables.")

    if v is None and t is None:
        v = math.sqrt(u**2 + 2*a*s)
        if a == 0:
            t = s / v
        else:
            t = (v - u) / a
    elif u is None and t is None:
        u = math.sqrt(v**2 - 2*a*s)
        if a == 0:
            t = s / v
        else:
            t = (v - u) / a
    elif a is None and t is None:
        a = (v**2 - u**2) / (2*s)
        if v == u:
            t = s / v
        else:
            t = (v - u) / a
    elif s is None and t is None:
        s = (v**2 - u**2) / (2*a) if a != 0 else u * t
        if a == 0:
            t = s / u
        else:
            t = (v - u) / a
    elif v is None and a is None:
        a = (2*(s - u*t)) / (t**2)
        v = u + a*t
    elif u is None and a is None:
        a = (2*(s - v*t)) / (-t**2)
        u = v - a*t 
    elif s is None and a is None:
        a = (v - u) / t
        s = (u + v) / 2 * t
    elif v is None and s is None:
        v = u + a*t
        s = u*t + 0.5*a*(t**2)
    elif u is None and s is None:
        u = v - a*t 
        s = u*t + 0.5*a*(t**2)
    elif a is None and s is None:
        a = (v - u) / t
        s = (u + v) / 2 * t

    return {'v': v, 'u': u, 'a': a, 's': s, 't': t}


#--------------------------------------------------------------------------------    
#As long as you’re consistent across all variables in a single calculation, it doesn’t matter whether you use:
#- meters, m/s, m/s²
#- feet, ft/s, ft/s²
#- miles, mph, mph/s

#v1 refers to the first vehicle we are computing
#v2 feeds off that 

#  u - intial velocity; v- final velocity; a - acceleration; s- displacement; t - Time elapsed

# 1 MPH = 1.46 ft/sec
 # For ranged lists --- arr_1d = np.arange(0, 10, 2).tolist() --- start, end, iteration


def t_d_single(u_1_list=[None],a_1_list=[None],s_1_list=[None],v_1_list=[None], t_1_list=[None]):

    
    
    
    results = []
    
    for u_1, a_1, s_1, v_1, t_1 in product(
        u_1_list, a_1_list, s_1_list, v_1_list, t_1_list):
        # Solve Honda
        v1 = all_eom(v=v_1, u=u_1, a=a_1, s=s_1, t=t_1)
    
    
    
        # Store result
        results.append({
            'u':v1['u'], 'v': v1['v'],'a': v1['a'], 's': v1['s'],  't': v1['t']
        })
    
    # Print
    #convert to pandas
    results_df = pd.DataFrame(results)
    
    return results_df

def t_d_double(u_1_list=[None],a_1_list=[None],s_1_list=[None],v_1_list=[None], t_1_list=[None], u_2_list=[None],a_2_list=[None],s_2_list=[None],v_2_list=[None], t_2_list=[None]):

    
    # -------------------------
    # Run and collect results
    # -------------------------
    
    results = []
    
    for u_1, a_1, s_1, v_1, t_1, u_2, a_2, s_2, v_2, t_2 in product(
        u_1_list, a_1_list, s_1_list, v_1_list, t_1_list,
        u_2_list, a_2_list, s_2_list, v_2_list, t_2_list):
        # Solve Honda
        v1 = all_eom(v=v_1, u=u_1, a=a_1, s=s_1, t=t_1)
    
        # Solve Rivian using Honda's time if available
        v2 = all_eom(v=v_2, u=u_2, a=a_2, s=s_2, t=t_2)

        
        # Store result
        results.append({
            'u_1': v1['u'], 'v_1': v1['v'],'a_1': v1['a'], 's_1': v1['s'],  't_1': v1['t'],
            'u_2': v2['u'], 'v_2': v2['v'], 'a_2': v2['a'], 's_2': v2['s'],  't_2': v2['t']
        })
    
        results_df = pd.DataFrame(results)
        # Print
        return results_df

def skidding_friction(u_1_list=[None],s_1_list=[None],v_1_list=[None], t_1_list=[None],meow=[0.5],acc_sign=-1):
    g=32.2  #32.2 ft/sec^2 or 9.8 m/s^2
    
    
    a_1_list = [item * acc_sign for item in meow]
     
        
        
    
    results = []
    
    for u_1, a_1, s_1, v_1, t_1 in product(
        u_1_list, a_1_list, s_1_list, v_1_list, t_1_list):
        # Solve Honda
        v1 = all_eom(v=v_1, u=u_1, a=a_1, s=s_1, t=t_1)
    
    
    
        # Store result
        results.append({
            'u_1':v1['u'], 'v_1': v1['v'],'a_1': v1['a'], 's_1': v1['s'],  't_1': v1['t']
        })
    
    # Print
    #convert to pandas
    results_df = pd.DataFrame(results)
    
    return results_df





def convert_to_feet_units(values, unit_type):
    """
    Converts speed, distance, or acceleration values to feet-based units.
    
    Parameters:
        values (float or list of floats): The value(s) to convert.
        unit_type (str): The unit of the input value(s). Supported:
            - Speed: 'mph', 'kmph', 'mps'
            - Distance: 'miles', 'meters', 'yards'
            - Acceleration: 'mps2', 'kmph2','fps2','g','-g',
    
    Returns:
        float or list of floats: Converted value(s) in ft/sec, feet, or ft/sec².
    """
    unit_type = unit_type.lower()

    # Conversion factors
    conversion_factors = {
        'mph': 1.46667,
        'kmph': 0.911344,
        'mps': 3.28084,
        'miles': 5280,
        'meters': 3.28084,
        'yards': 3,
        'mps2': 3.28084,
        'kmph2': 0.000084645,
        'g':32.2,
        '-g':-32.2,
        'feet': 1,
        'fps': 1,
        'fps2':1,
	    'seconds':1,
	    'minutes':60,
        'hours':3600
    }

    if unit_type not in conversion_factors:
        raise ValueError(f"Unsupported unit type: {unit_type}")

    factor = conversion_factors[unit_type]

    # Handle single value or list

    if isinstance(values, list):
        return [v * factor if v is not None else None for v in values]
    else:
        return values * factor if values is not None else None

def convert_from_feet_units(values, target_unit):
    """
    Converts speed, distance, or acceleration values from feet-based units to the specified target unit.

    Parameters:
        values (float or list of floats): The value(s) in feet, ft/s, or ft/s².
        target_unit (str): The desired output unit. Supported:
            - Speed: 'mph', 'kmph', 'mps'
            - Distance: 'miles', 'meters', 'yards', 'inches', 'centimeters'
            - Acceleration: 'mps2', 'kmph2'
            - Time: 'seconds', 'minutes', 'hours'

    Returns:
        float or list of floats: Converted value(s) in the target unit.
    """
    target_unit = target_unit.lower()

    # Conversion factors FROM feet-based units
    conversion_factors = {
        'mph': 1 / 1.46667,
        'kmph': 1 / 0.911344,
        'mps': 1 / 3.28084,
        'miles': 1 / 5280,
        'meters': 1 / 3.28084,
        'yards': 1 / 3,
        'mps2': 1 / 3.28084,
        'kmph2': 1 / 0.000084645,
        'g':1/32.2,
        '-g':-1/32.2,
        'feet': 1,
        'fps': 1,
        'fps2': 1,
        'seconds':1,
        'minutes':1/60,
        'hours':1/3600
    }

    if target_unit not in conversion_factors:
        raise ValueError(f"Unsupported target unit: {target_unit}")

    factor = conversion_factors[target_unit]

    if isinstance(values, list):
        return [v * factor if v is not None else None for v in values]
    else:
        return values * factor if values is not None else None

def safe_cast(val):
    if val is None or val.strip().lower() == 'none' or val.strip() == '':
        return [None]
    try:
        result = eval(val, {"np": np, "__builtins__": {}})
        if isinstance(result, (list, np.ndarray)):
            return list(result)
        else:
            return [result]
    except:
        return [None]

def safe_castv2(val):
    if val is None or val.strip().lower() == 'none' or val.strip() == '':
        return [None]
    try:
        result = eval(val, {"np": np, "__builtins__": {}})
        if isinstance(result, (list, np.ndarray)):
            return [float(x) if isinstance(x, (np.integer, np.floating)) else x for x in result]
        else:
            return [float(result) if isinstance(result, (np.integer, np.floating)) else result]
    except:
        return [None]

def safe_castv3(val):
    if val is None or val.strip().lower() == 'none' or val.strip() == '':
        return [None]
    try:
        result = eval(val, {"np": np, "__builtins__": {}})
        if isinstance(result, (list, np.ndarray)):
            return list(result)
        elif isinstance(result, tuple):
            return list(result)
        else:
            return [result]
    except:
        return [None]


In [3]:

def normalize(val):
    if isinstance(val, str):
        return safe_castv3(val)
    else:
        # already a list or number
        return val

# Define unit options
distance_units = ['meters', 'yards', 'miles','feet']
speed_units = ['mps', 'kmph', 'mph', 'fps']
acceleration_units = ['mps2', 'kmph2','fps2','g','-g']
time_units=['seconds', 'minutes', 'hours']

# ---- New: input mode options ----
entry_modes = ['Manual / Comma / Bracket', 'NumPy Range', 'NumPy Linspace']

# ---- Helpers to create a mode-controlled input UI for each variable ----
def make_mode_input(label, initial_text=''):
    mode_dd = widgets.Dropdown(options=entry_modes, value='Manual / Comma / Bracket',
                               description=f'{label} Mode:', style={'description_width': 'initial'})

    # Manual / comma / bracket text field
    manual_text = widgets.Text(value=initial_text, description=f'{label}:', style={'description_width': 'initial'})

    # NumPy Range widgets
    r_start = widgets.FloatText(value=0.0, description='Start:', style={'description_width': 'initial'})
    r_end   = widgets.FloatText(value=10.0, description='End:',   style={'description_width': 'initial'})
    r_step  = widgets.FloatText(value=1.0, description='Step:',  style={'description_width': 'initial'})

    # NumPy Linspace widgets
    l_start  = widgets.FloatText(value=0.0, description='Start:', style={'description_width': 'initial'})
    l_end    = widgets.FloatText(value=1.0, description='End:',   style={'description_width': 'initial'})
    l_points = widgets.IntText(value=5, description='Points:',    style={'description_width': 'initial'})

    # Container to swap the input controls based on mode
    input_container = widgets.VBox()

    def update_inputs(_=None):
        m = mode_dd.value
        if m == 'Manual / Comma / Bracket':
            input_container.children = [manual_text]
        elif m == 'NumPy Range':
            input_container.children = [widgets.HBox([r_start, r_end, r_step])]
        else:  # NumPy Linspace
            input_container.children = [widgets.HBox([l_start, l_end, l_points])]
    mode_dd.observe(update_inputs, names='value')
    update_inputs()

    # Getter that returns the parsed Python value (scalar or list)
    def getter():
        m = mode_dd.value
        try:
            if m == 'Manual / Comma / Bracket':
                txt = manual_text.value.strip()
                if txt == '':
                    return ''
                # Accept bracketed lists, single scalars, or comma lists without brackets
                # If plain comma list without brackets, wrap in brackets before safe_cast
                stripped = txt.replace(' ', '')
                if ',' in stripped and not (txt.startswith('[') and txt.endswith(']')):
                    return safe_castv3(f'[{txt}]')
                return safe_castv3(txt)
            elif m == 'NumPy Range':
                start = float(r_start.value)
                end   = float(r_end.value)
                step  = float(r_step.value)
                if step == 0:
                    return []
                return np.arange(start, end+step, step).tolist()
            else:  # NumPy Linspace
                start  = float(l_start.value)
                end    = float(l_end.value)
                points = int(l_points.value)
                if points <= 0:
                    return []
                return np.linspace(start, end, points).tolist()
        except Exception:
            return ''

    # Return the UI row pieces so we can place them next to unit dropdowns
    return mode_dd, input_container, getter, manual_text, (r_start, r_end, r_step), (l_start, l_end, l_points)

# ---- Build inputs (with modes) and units on the Calculate tab ----
# Distance
dist_mode, dist_input_box, get_distance, _, _, _ = make_mode_input('Distance', initial_text='[10,1]')
distance_unit = widgets.Dropdown(options=distance_units, value='feet', description='Distance Unit:', style={'description_width': 'initial'})
dist_row = widgets.HBox([dist_input_box, distance_unit, dist_mode])

# Initial speed
u_mode, u_input_box, get_u, _, _, _ = make_mode_input('Initial Speed', initial_text='16')
initial_speed_unit = widgets.Dropdown(options=speed_units, value='fps', description='Initial Speed Unit:', style={'description_width': 'initial'})
u_row = widgets.HBox([u_input_box, initial_speed_unit, u_mode ])

# Final speed
v_mode, v_input_box, get_v, _, _, _ = make_mode_input('Final Speed', initial_text='26,36')
final_speed_unit = widgets.Dropdown(options=speed_units, value='fps', description='Final Speed Unit:', style={'description_width': 'initial'})
v_row = widgets.HBox([v_input_box, final_speed_unit, v_mode])

# Time
t_mode, t_input_box, get_t, _, _, _ = make_mode_input('Time', initial_text='')
time_unit = widgets.Dropdown(options=time_units, value='seconds', description='Time Unit:', style={'description_width': 'initial'})
t_row = widgets.HBox([ t_input_box, time_unit, t_mode])

# Acceleration
a_mode, a_input_box, get_a, _, _, _ = make_mode_input('Acceleration', initial_text='')
acceleration_unit = widgets.Dropdown(options=acceleration_units, value='fps2', description='Acceleration Unit:', style={'description_width': 'initial'})
a_row = widgets.HBox([ a_input_box, acceleration_unit, a_mode])

# Output unit selectors (unchanged)
o_distance_unit = widgets.Dropdown(description="Distance:", options=distance_units, value='feet')
o_initial_speed_unit = widgets.Dropdown(description="Initial Speed:", options=speed_units, value='fps')
o_final_speed_unit = widgets.Dropdown(description="Final Speed:", options=speed_units, value='fps')
o_time_unit = widgets.Dropdown(description="Time:", options=time_units, value='seconds')
o_acceleration_unit = widgets.Dropdown(description="Acceleration:",options=acceleration_units, value='fps2')

# Submit button and output
submit_btn = widgets.Button(description='Submit', button_style='success')
output = widgets.Output()

def on_submit(b):
    output.clear_output()
    with output:
        # Use mode-driven getters instead of raw text
        s_raw = get_distance()
        u_raw = get_u()
        v_raw = get_v()
        t_raw = get_t()
        a_raw = get_a()

        s = normalize(s_raw)
        u = normalize(u_raw)
        v = normalize(v_raw)
        t = normalize(t_raw)
        a = normalize(a_raw)

        print("\nYour Inputs \n")
        print("Distance (s):", s)
        print("Initial Speed (u):", u)
        print("Final Speed (v):", v)
        print("Time (t):", t)
        print("Acceleration (a):", a)

        # Convert to internal feet-based units
        s = convert_to_feet_units(s, distance_unit.value)
        u = convert_to_feet_units(u, initial_speed_unit.value)
        v = convert_to_feet_units(v, final_speed_unit.value)
        t = convert_to_feet_units(t, time_unit.value)
        a = convert_to_feet_units(a, acceleration_unit.value)

        # Compute
        df = t_d_single(u_1_list=u, a_1_list=a, s_1_list=s, v_1_list=v, t_1_list=t)

        
       
        # Convert outputs to selected output units
        df['s'] = convert_from_feet_units(df['s'], o_distance_unit.value)
        df['u'] = convert_from_feet_units(df['u'], o_initial_speed_unit.value)
        df['v'] = convert_from_feet_units(df['v'], o_final_speed_unit.value)
        df['t'] = convert_from_feet_units(df['t'], o_time_unit.value)
        df['a'] = convert_from_feet_units(df['a'], o_acceleration_unit.value)

        print("\n Outputs \n")
        display(df)

        return s, u, v, t, a, df

submit_btn.on_click(on_submit)

# Layout (Time-Dist. Calculate tab) — units on left, mode dropdown and input controls to the right
rows = [
    widgets.HBox([widgets.Label(value="Insert 3 known Values, other 2 are unknown. Choose mode and enter values")]),
    dist_row,
    u_row,
    v_row,
    t_row,
    a_row
]

o_rows= [
    widgets.HBox([widgets.Label(value="Select Unit for Outputs of Calculations")]),
    widgets.HBox([o_distance_unit]),
    widgets.HBox([o_initial_speed_unit]),
    widgets.HBox([o_final_speed_unit]),
    widgets.HBox([o_time_unit]),
    widgets.HBox([o_acceleration_unit])
]

html_text = """<pre style="white-space:pre-wrap">
Data Entry Guide (Now supports modes)

Modes (per field, right of unit dropdown):
 - Manual / Comma / Bracket: type a single number, a comma list i.e. 10,20,30, or a bracketed list i.e. [10,20,30]
 - NumPy Range: specify Start, End, Step (creates list via np.arange)
 - NumPy Linspace: specify Start, End, Points (creates list via np.linspace)

Units
Velocity: fps (Feet Per Second), mph (Miles Per Hour), mps (Meter Per Second), kmph (Kilometer Per Hour)
Acceleration: fps2 (Feet Per Second Squared), mps2 (Meter Per Second Squared), kmph2 (Kilometer Per Hour Squared), g (Gravitational Acc)
Default outputs are to feet, fps, fps2 and seconds

Importing Spreadsheets
This porgram accepts comma seaprated lists. Convert an excel column, row or any range to a comma separated list with excel command:
=TEXTJOIN(",",TRUE,A1:A5)
This would turn the column A from rows 1 to 5 into  a comma separated list you can paste in here
</pre>"""

Help = widgets.HTML(value=html_text)

Settings=widgets.VBox(o_rows)
Calculate = widgets.VBox(rows + [widgets.HBox([submit_btn]), output])

tab_nest = widgets.Tab()
tab_nest.children = [Calculate, Settings, Help]
tab_nest.set_title(0, 'Time-Dist. Calculate')
tab_nest.set_title(1, 'Output Units')
tab_nest.set_title(2, 'Help')

display(tab_nest)

In [4]:
s, u, v, t, a, df = on_submit(0)
# Create checkboxes for each column
column_checkboxes = [widgets.Checkbox(value=False, description=col) for col in df.columns]
column_box = widgets.VBox(column_checkboxes)

# Create checkboxes for export formats
export_options = ['CSV', 'Histogram', 'Comma-separated list']
export_checkboxes = [widgets.Checkbox(value=False, description=opt) for opt in export_options]
export_box = widgets.VBox(export_checkboxes)

# Bin slider for histogram
bin_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='Bins:',
    style={'description_width': 'initial'}
)

# Export button
export_button = widgets.Button(description='Export', button_style='success')

# Output area
output_area = widgets.Output()

# Callback function for export
def export_callback(b):
    with output_area:
        clear_output()
        selected_cols = [cb.description for cb in column_checkboxes if cb.value]
        selected_exports = [cb.description for cb in export_checkboxes if cb.value]
        bins = bin_slider.value

        if not selected_cols:
            print("⚠️ Please select at least one column.")
            return
        if not selected_exports:
            print("⚠️ Please select at least one export format.")
            return

        selected_df = df[selected_cols]

        if 'CSV' in selected_exports:
            filename = 'exported_data.csv'
            selected_df.to_csv(filename, index=False)
            display(FileLink(filename))
            print(f"✅ CSV exported to '{filename}'")

        if 'Histogram' in selected_exports:
            os.makedirs("histograms", exist_ok=True)
            for col in selected_cols:
                data = df[col].dropna()
                counts, edges = np.histogram(data, bins=bins)

                # Save histogram image
                plt.figure()
                plt.hist(data, bins=bins)
                plt.title(f'Histogram of {col}')
                plt.xlabel(col)
                plt.ylabel('Frequency')
                plt.grid(True)
                img_path = f"histograms/{col}_histogram.png"
                plt.savefig(img_path)
                plt.close()

                # Save histogram data as CSV
                hist_df = pd.DataFrame({
                    'Bin Start': edges[:-1],
                    'Bin End': edges[1:],
                    'Count': counts
                })
                csv_path = f"histograms/{col}_histogram_data.csv"
                hist_df.to_csv(csv_path, index=False)
                

                display(FileLink(csv_path))
                display(FileLink(img_path))

                print(f"📊 Histogram for '{col}' saved as image and CSV in 'histograms/'")

        if 'Comma-separated list' in selected_exports:
            for col in selected_cols:
                values = ', '.join(map(str, df[col].tolist()))
                print(f"{col}: {values}")



def query_callback(b):
    with query_output:
        clear_output()
        selected_cols = [cb.description for cb in column_checkboxes if cb.value]
        if not selected_cols:
            print("⚠️ Please select at least one column.")
            return

        custom_percentile = percentile_input.value
        include_ci = ci_checkbox.value  # ✅ Checkbox value

        print(f"📊 Querying statistics for selected columns with custom percentile: {custom_percentile}%\n")

        for col in selected_cols:
            data = df[col].dropna()
            n = len(data)
            if not np.issubdtype(data.dtype, np.number):
                print(f"❌ Column '{col}' is not numeric. Skipping.\n")
                continue

            mean = data.mean()
            median = data.median()
            std = data.std()
            var = data.var()
            data_range = data.max() - data.min()
            mode = data.mode().tolist()
            q25 = np.percentile(data, 25)
            q75 = np.percentile(data, 75)
            custom_q = np.percentile(data, custom_percentile)

            print(f"📈 Column: {col}")
            print(f"  Count (n): {n}")
            print(f"  Mean: {mean:.2f}")
            print(f"  Median: {median:.2f}")
            print(f"  Standard Deviation: {std:.2f}")
            print(f"  Variance: {var:.2f}")
            print(f"  Range (max - min): {data_range:.2f}")
            print(f"  Mode: {mode}")
            print(f"  25th Percentile: {q25:.2f}")
            print(f"  75th Percentile: {q75:.2f}")
            print(f"  {custom_percentile}th Percentile: {custom_q:.2f}")

            # ✅ Confidence Interval section
            if include_ci:
                if n < 10:
                    print(f"  ⚠️ Too few samples ({n}) for a reliable confidence interval.")
                else:
                    sem = stats.sem(data)
                    ci = stats.t.interval(0.95, df=n-1, loc=mean, scale=sem)
                    print(f"  95% Confidence Interval for Mean: [{ci[0]:.2f}, {ci[1]:.2f}]")

            # 🧭 “What You Should Know” diagnostic section
            print("\n  🧭 What You Should Know:")
            if n < 10:
                print("   • ⚠️ Very small sample — interpret all stats with extreme caution.")
            elif n < 30:
                print("   • ⚠️ Small sample — estimates may vary widely if data changes.")
            elif n < 100:
                print("   • ✅ Moderate reliability — results are fairly stable.")
            else:
                print("   • 🟢 Large dataset — results are statistically robust.")

            print("\n" + "-"*60 + "\n")

# Custom percentile input
percentile_input = widgets.BoundedFloatText(
    value=50.0,
    min=0.0,
    max=100.0,
    step=0.1,
    description='Custom Percentile:',
    style={'description_width': 'initial'}
)

ci_checkbox = widgets.Checkbox(
    value=False,
    description='Include 95% Confidence Intervals',
    style={'description_width': 'initial'}
)

twoopts=widgets.HBox([percentile_input,ci_checkbox])

export_button.on_click(export_callback)

# Export tab
exp_box = widgets.VBox([
    widgets.Label("✅ Select columns to export:"),
    column_box,
    widgets.Label("📦 Choose export formats:"),
    export_box,
    bin_slider,
    export_button,
    output_area
])

# Query button
query_button = widgets.Button(description='Query Selected Columns', button_style='info')

# Output area for query results
query_output = widgets.Output()

query_button.on_click(query_callback)

# Query tab
query_box = widgets.VBox([
    widgets.Label("🔍 Select columns to query:"),
    column_box,
    twoopts,
    query_button,
    query_output
])

# Tabs
tab2_nest = widgets.Tab()
tab2_nest.children = [query_box, exp_box]
tab2_nest.set_title(0, 'Statistical Query')
tab2_nest.set_title(1, 'Export Data')
tab2_nest
